# 为机器学习初学者而建的手写数据预测练习

这篇教程主要面向机器学习初学者，如果你已经了解 MNIST 数据集以及 softmax （多分类 logistic 回归）的原理，你可以选择阅读更快节奏的教程。

## MNIST 数据集——手写体识别

MNIST 数据集是 Yann LeCun 收集的手写体图片以及相关的标签。为了简化整个教程，tensorflow 已经把 MNIST 整合到教程模块了，读者可以直接运行如下代码，tensorflow 将为你自动下载相关的数据集。

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


下载的数据将被分为 3 个部分：其中 55,000 条记录作为训练数据（`mnist.train`）、10,000 条记录作为测试数据（`mnist.test`），以及 5,000 条记录作为验证数据（`mnist.validation`）。每个数据集有两个部分：图片（xs）以及类标（ys）。这个任务就是给定图片的情况下，模型给出它的类标。

每张图片都是 28x28 的矩阵，例如：

![图片矩阵的转化方式](https://www.tensorflow.org/versions/r0.8/images/MNIST-Matrix.png)

我们可以把这个 28x28=784 的矩阵平整为一个数组，这个数组里的数即这个像素的颜色深度（这里用的是灰度）。从这个角度看，MNIST 数据集实际上提供了一些 784 维的数组，以及类标。

![对 MNIST 数据训练数据处理方法的图示](https://www.tensorflow.org/versions/r0.8/images/mnist-train-xs.png)

MNIST 的类标由 0 到 9 组成，描述了每张手写体数字的意义。我们将采用 独热编码（one-hot encoder）来建立一个多分类问题。独热编码就是把 n 位取值的数变为 n 维向量，其中每一维代表了一种取值的情况。例如，有一个变量有 3 中取值的情况，则设一个 3 维向量，(1, 0, 0)、(0, 1, 0)、(0, 0, 1) 分别代表一种取值情况。

![对 MNIST 数据集中类标处理方法的图示](https://www.tensorflow.org/versions/r0.8/images/mnist-train-ys.png)

## 模型的建立

引入 tensorflow ：

In [2]:
import tensorflow as tf

定义待优化的参数

In [3]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

定义特征 x 以及标签 y，作为 placeholder，即参数。

In [4]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

使用 softmax 方法定义目标函数。其中 softmax 是 logistic 函数的多分类扩展，表达式为：

$$  \begin{align} h_\theta(x^{(i)}) = \begin{bmatrix} p(y^{(i)} = 1 | x^{(i)}; \theta) \\ p(y^{(i)} = 2 | x^{(i)}; \theta) \\ \vdots \\ p(y^{(i)} = k | x^{(i)}; \theta) \end{bmatrix} = \frac{1}{ \sum_{j=1}^{k}{e^{ \theta_j^T x^{(i)} }} } \begin{bmatrix} e^{ \theta_1^T x^{(i)} } \\ e^{ \theta_2^T x^{(i)} } \\ \vdots \\ e^{ \theta_k^T x^{(i)} } \\ \end{bmatrix} \end{align} $$

输出的值为属于各个分类的概率大小。在 tensorflow 中，softmax 函数已经被封装好了。

In [5]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

使用交叉熵 $e = \sum_i \hat{y_i} \times \log y_i$ 作为损失函数。在本例中，

$$ \begin{align} J(\theta) &= -\frac{1}{m} \left[ \sum_{i=1}^m (1-y^{(i)}) \log (1-h_\theta(x^{(i)})) + y^{(i)} \log h_\theta(x^{(i)}) \right] \\ &= - \frac{1}{m} \left[ \sum_{i=1}^{m} \sum_{j=0}^{1} 1\left\{y^{(i)} = j\right\} \log p(y^{(i)} = j | x^{(i)} ; \theta) \right] \end{align}  $$

对于二分类， $h_\theta(x)$ 即 logistic 函数：

$$ \begin{align} h_\theta(x) = \frac{1}{1+\exp(-\theta^Tx)} \end{align} $$

对于多分类， $h_\theta(x)$ 即 softmax 函数在某个分类下的概率（见上一段 softmax 的表达式）。

In [6]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

使用梯度下降算法优化损失函数。tensorflow 为常见的优化方法提供了现成的封装函数。

In [7]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

初始化 tensorflow 变量，为后续运算做准备。

In [8]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())

这里采用随机训练，即每次随机取 100 个样本去优化损失函数，从而达到“学习”的目的。

In [9]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

## 模型评估

计算训练误差，tensorflow 是一个“张量”“流”系统，因此，在执行 `session.run()` 方法时，所有相关的张量都已经被计算出来了。这里可以直接进行比较。

和 NumPy 类似，tensorflow 提供了 `equal()` 方法对给定的数组中每个元素进行相等判断。而 `argmax` 则是选择一个数组中的最大值对应的下标。

In [10]:
correct_prediction = tf.equal(
    tf.argmax(y, 1),
    tf.argmax(y_, 1)
)
accuracy = tf.reduce_mean(
    tf.cast(correct_prediction, tf.float32)
)
print(sess.run(accuracy, feed_dict={
            x: mnist.test.images,
            y_: mnist.test.labels
        }))

0.9157
